- np.concatenate((a,b,c...))<br\>
- np.append(arr,values)<br\>
- df.columns.map(func)<br\>
- df.columns.append()<br\>
- df.append()

In [1]:
 #首先要引入各个包
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib
import os
%matplotlib inline

In [2]:
def select_col(fn,is_print=False):
    df = pd.read_excel(fn)
    columns = ['AST', 'BLK', 'BLKA', 'DREB', 'FG3A', 'FG3M', 'FGA', 'FGM', 
               'FTA','FTM','OREB', 'PF', 'PFD', 'PTS','PTS_2ND_CHANCE', 'PTS_FB', 
               'PTS_OFF_TOV', 'PTS_PAINT', 'REB','STL', 'TEAM_ABBREVIATION', 'TOV','GAME_ID']
    return df[columns]

df = pd.DataFrame()
for fn in os.listdir('../数据收集'):
    if fn.endswith('.xlsx') and "games" not in fn:
        new_df = select_col('../数据收集/'+fn)
        df = pd.concat([df,new_df])
df.head(10)

AST  BLK  BLKA  DREB  FG3A  FG3M  FGA  FGM  FTA  FTM    ...     PTS  \
0   19    4     7    29    29    11   72   30   19   15    ...      86   
1   30    4     1    39    27     7   79   41   18   14    ...     103   
2   31    4     3    35    31     9   94   44   37   29    ...     126   
3   39    3     3    36    37    16   89   45    8    8    ...     114   
4   26    4     4    39    23    11   89   44   34   24    ...     123   
5   19    4     6    36    24     7   80   30   23   15    ...      82   
6   25    1     4    34    28    11   90   40   17   13    ...     104   
7   19    3     3    32    18     9   80   34   26   22    ...      99   
8   24    6     6    42    27     6   85   34   29   21    ...      95   
9   16    3     7    32    27     5   98   34   32   19    ...      92   

   PTS_2ND_CHANCE  PTS_FB  PTS_OFF_TOV  PTS_PAINT  REB  STL  \
0               3       0            3         30   37    5   
1               9      14           14         50   50    6   
2               6       9           20         58   45    8   
3               7      16           15         42   42    7   
4              25      17           14         44   52    7   
5               6      15           28         34   44    9   
6              17      17           15         46   45    4   
7              21       0           10         38   47    7   
8               8       7           12         36   51    5   
9              30      20           21         46   58   10   

   TEAM_ABBREVIATION  TOV   GAME_ID  
0                ATL   20  21601226  
1                ATL   17  21601212  
2                ATL   12  21601197  
3                ATL   11  21601179  
4                ATL   17  21601177  
5                ATL   20  21601148  
6                ATL   13  21601138  
7                ATL   17  21601111  
8                ATL   19  21601106  
9                ATL   21  21601088  

[10 rows x 23 columns]

In [3]:
games = sorted(df['GAME_ID'].unique())
group_df = df.set_index('TEAM_ABBREVIATION').groupby('GAME_ID')

In [4]:
df.columns

Index(['AST', 'BLK', 'BLKA', 'DREB', 'FG3A', 'FG3M', 'FGA', 'FGM', 'FTA',
       'FTM', 'OREB', 'PF', 'PFD', 'PTS', 'PTS_2ND_CHANCE', 'PTS_FB',
       'PTS_OFF_TOV', 'PTS_PAINT', 'REB', 'STL', 'TEAM_ABBREVIATION', 'TOV',
       'GAME_ID'],
      dtype='object')

In [5]:
example_group = group_df.get_group(games[0])
columns = example_group.loc[example_group.index[0]].index
columns = columns.append(columns.map(lambda x:"OPP_"+x))
columns

Index(['AST', 'BLK', 'BLKA', 'DREB', 'FG3A', 'FG3M', 'FGA', 'FGM', 'FTA',
       'FTM', 'OREB', 'PF', 'PFD', 'PTS', 'PTS_2ND_CHANCE', 'PTS_FB',
       'PTS_OFF_TOV', 'PTS_PAINT', 'REB', 'STL', 'TOV', 'GAME_ID', 'OPP_AST',
       'OPP_BLK', 'OPP_BLKA', 'OPP_DREB', 'OPP_FG3A', 'OPP_FG3M', 'OPP_FGA',
       'OPP_FGM', 'OPP_FTA', 'OPP_FTM', 'OPP_OREB', 'OPP_PF', 'OPP_PFD',
       'OPP_PTS', 'OPP_PTS_2ND_CHANCE', 'OPP_PTS_FB', 'OPP_PTS_OFF_TOV',
       'OPP_PTS_PAINT', 'OPP_REB', 'OPP_STL', 'OPP_TOV', 'OPP_GAME_ID'],
      dtype='object')

In [6]:
feature_df = pd.DataFrame()
for game in games:
    game_df = group_df.get_group(game)
    one = game_df.loc[game_df.index[0]]
    two = game_df.loc[game_df.index[1]]
    feature_df = feature_df.append(pd.DataFrame(np.concatenate((one.values,two.values)),index=columns,columns = [game]).T)
    feature_df = feature_df.append(pd.DataFrame(np.concatenate((two.values,one.values)),index=columns,columns = [game]).T)
feature_df.head()

AST  BLK  BLKA  DREB  FG3A  FG3M  FGA  FGM  FTA  FTM     ...       \
21600001   31    5     6    40    35    13   94   45   19   14     ...        
21600001   17    6     5    29    27     9   87   32   20   15     ...        
21600002   22    3     5    29    19    13   75   39   22   22     ...        
21600002   19    5     3    25    24     8   82   40   16   16     ...        
21600003   24    6     3    27    33     7   85   40   18   13     ...        

          OPP_PFD  OPP_PTS  OPP_PTS_2ND_CHANCE  OPP_PTS_FB  OPP_PTS_OFF_TOV  \
21600001       22       88                  16           9               14   
21600001       22      117                  16          16               21   
21600002       18      104                   4           8                8   
21600002       19      113                  15           4               20   
21600003       19      129                  26          24               17   

          OPP_PTS_PAINT  OPP_REB  OPP_STL  OPP_TOV  OPP_GAME_ID  
21600001             28       42        6       18     21600001  
21600001             52       51       12       15     21600001  
21600002             50       31        9       14     21600002  
21600002             34       34        5       13     21600002  
21600003             50       55       13       14     21600003  

[5 rows x 44 columns]

In [7]:
del feature_df['GAME_ID']
del feature_df['OPP_GAME_ID']
feature_df.to_excel('features.xlsx')